# Неточности в задании (оценка не снижалась)

В задании, где нужно найти ошибки стемминга, когда слова не изменяются после стемминга не очень корректная формулировка - нужны именно ошибки, когда слово не изменилось после стеминга, а должно было! Часто приводились слова, которые стемить не нужно. 

Во втором задании сказано "Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize. ". Тут есть ошибка в порядке предобработки - разбивать на предложения нужно в самом начале, т.к. без регистра и пунктуации будут получаться плохие предложения.

In [67]:
# пример с сентенайзером вначале

In [58]:
text = 'Во втором задании сказано "Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize. ". Тут есть ошибка в порядке предобработки - разбивать на предложения нужно в самом начале, т.к. без регистра и пунктуации будут получаться плохие предложения. '

In [61]:
from razdel import sentenize
[sent.text for sent in list(sentenize(text))]

['Во втором задании сказано "Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize.',
 '".',
 'Тут есть ошибка в порядке предобработки - разбивать на предложения нужно в самом начале, т.к. без регистра и пунктуации будут получаться плохие предложения.']

In [68]:
# пример с сентенайзером в порядке по заданию

In [69]:
from razdel import sentenize
from string import punctuation
import re
text = text.lower()
text = re.sub(f'[{punctuation}]', '', text)
[sent.text for sent in list(sentenize(text))]

['во втором задании сказано приведите очищенный текст к нижнему регистру удалите все знаки пунктуации разделите на предложения библиотекой rsekeze токенизируйте библиотекой rzdelkeze  тут есть ошибка в порядке предобработки  разбивать на предложения нужно в самом начале тк без регистра и пунктуации будут получаться плохие предложения']

## Фидбек по коду (не влияет на оценку)

Питон удобный язык, но не самый быстрый, поэтому нужно следить за эффективностью кода. Хотя бы на базовом уровне.

Вот несколько советов:

    Циклы - самый большой источник неэффективности в вашем коде. Используйте минимальное возможное количество циклов.  Старайтесь     никогда не использовать вложенные циклы;
    
    Изучите базовые структуры данных питона (их слабые и сильные стороны)

# Примеры

Напишем функцию, которая генерирует рандомное слово из заданного набора символов. Сделаем два списка и попробуем проверить сколько одинаковых слов в этих списках.

In [6]:
from string import ascii_letters
from numpy.random import choice

In [12]:
def generate_random_word(letters, n):
    word = ''.join(choice(list(letters), n))
    return word

In [13]:
generate_random_word(ascii_letters, 5)

'dfSVh'

In [27]:
list_1 = [generate_random_word(ascii_letters, 3) for i in range(10000)]
list_2 = [generate_random_word(ascii_letters, 3) for i in range(10000)]

### ПЛОХОЙ ПРИМЕР

In [28]:
%%time
intersection = []

for word1 in list_1:
    for word2 in list_2:
        if word1 == word2:
            intersection.append(word1)


CPU times: user 4.83 s, sys: 7.43 ms, total: 4.84 s
Wall time: 4.84 s


### Получше, но все равно плохо

In [29]:
%%time
intersection = []

for word1 in list_1:
    if word1 in list_2: # Список не подходит для оператора in
        intersection.append(word1)

CPU times: user 1.2 s, sys: 4.05 ms, total: 1.2 s
Wall time: 1.2 s


### Получше, но все еще с циклом 

In [30]:
%%time
intersection = []

# преобразуем список в множество
set_2 = set(list_2)

for word1 in list_1:
    if word1 in set_2:
        intersection.append(word1)

CPU times: user 1.29 ms, sys: 11 µs, total: 1.3 ms
Wall time: 1.3 ms


### Совсем без цикла

In [31]:
%%time
# между множествами можно посчитать пересечение
intersection = set(list_1) & set(list_2)

CPU times: user 1.04 ms, sys: 33 µs, total: 1.07 ms
Wall time: 1.08 ms


In [32]:
len(intersection)

666

Теперь давайте для каждого слова найдем его частоту в первом и втором списках.

### Плохой пример

У списка конечно есть метод count, но лучше им не пользоваться (по крайней мере в такой задаче).

In [35]:
%%time
vocab2count = {}

for word in list_1:
    vocab2count[word] = [list_1.count(word), list_2.count(word)]

for word in list_2:
    vocab2count[word] = [list_1.count(word), list_2.count(word)]

CPU times: user 5.12 s, sys: 9.77 ms, total: 5.13 s
Wall time: 5.13 s


### Встроенный счетчик

Для частотных словарей удобно использовать Counter. Он оптимизирован под сбор частотностей, поэтому лучше всегда пользоваться им, а не писать свое

In [38]:
from collections import Counter

In [39]:
%%time
vocab2count = {}

vocab1 = Counter(list_1)
vocab2 = Counter(list_2)
vocab = set(vocab1.keys()) | set(vocab2.keys())

for word in vocab:
    vocab2count[word] = [vocab1[word], vocab2[word]]
    

CPU times: user 86.6 ms, sys: 16.4 ms, total: 103 ms
Wall time: 102 ms


Теперь давайте переопределим второй список - там будут первые символы слов из первого списка. Теперь сделаем маппинг первая буква: все слова, которые с нее начинаются. (Это похоже на задание с поиском ошибок в стемминге)

In [40]:
list_1 = [generate_random_word(ascii_letters, 3) for i in range(10000)]
list_2 = [word[:1] for word in list_1]

### defaultdict

Для этого хорошо подходит defaultdict - словарь с дефолтным значением

In [41]:
from collections import defaultdict

In [52]:
%%time
mapping = defaultdict(list)

for word, letter in zip(list_1, list_2): # zip позволяет итерироваться по двум спискам одновременно
    mapping[letter].append(word)

CPU times: user 2.01 ms, sys: 5 µs, total: 2.02 ms
Wall time: 2.02 ms


In [53]:
%%time
# чтобы было понятно чем удобнее вот пример без него


mapping = {}

for word, letter in zip(list_1, list_2):
    if letter in mapping:
        mapping[letter].append(word)
    else:
        mapping[letter] = [word]
    

CPU times: user 2.8 ms, sys: 47 µs, total: 2.85 ms
Wall time: 2.85 ms


In [51]:
mapping['J'][:10]

['JNk', 'Jac', 'JZN', 'JyW', 'JPP', 'JIq', 'JlL', 'JGy', 'JtJ', 'Jev']

### lambda, map, filter не улучшают эффективность, но сильно снижают читаемость

Старайтесь не пользоваться lambda, map, filter вообще 

Допустим нам надо выбрать слова из словаря, встречающиеся больше 2 раз и привезти их к нижнему регистру. 

### Плохой пример с lambda map filter

In [88]:
%%time
lower_frequent = list(map(lambda x: x[0].lower(), filter(lambda x: x[1] > 2, vocab1.most_common())))

CPU times: user 3.06 ms, sys: 9 µs, total: 3.07 ms
Wall time: 3.07 ms


In [89]:
repeating_sentences

['auk', 'tbw', 'ndc', 'tzl', 'fjs', 'sco']

### Лучше вот так написать

По времени тут нет различия, но такой вариант намного более читаемый

In [90]:
%%time
[word.lower() for word, count in vocab1.most_common() if count > 2]

CPU times: user 2.4 ms, sys: 15 µs, total: 2.42 ms
Wall time: 2.42 ms


['auk', 'tbw', 'ndc', 'tzl', 'fjs', 'sco']

### Не вызывайте два раза то, что можно сохранить в переменную

In [92]:
# код не запускается, но должно быть читаемо
lemmas = list()

for sentence in sentences:
    for token in sentence:
        if mystem.lemmatize(token)[0] != morph.parse(token)[0].normal_form:
            lemmas.append((token, morph.parse(token)[0].normal_form, mystem.lemmatize(token)[0]))

Вот тут лемматизация сначала вызывается для сравнения, а потом для добавления в список. Можно сохранить в переменную и переиспользовать.

In [ ]:
# код не запускается, но должно быть читаемо
lemmas = list()

for sentence in sentences:
    for token in sentence:
        lemma_pymorphy = morph.parse(token)[0].normal_form
        lemma_mystem = mystem.lemmatize(token)[0]
        if lemma_pymorphy != lemma_mystem:
            lemmas.append((token, lemma_mystem, lemma_pymorphy))

Еще есть моржовый оператор, пример тут - https://habr.com/ru/company/otus/blog/472432/